# The Simplest Possible Hierarchical Inference Problem, Gibbs Sampled

There's a set of $N$ objects described by one variable x, generated from a Gaussian distribution. For each object we have a measurement of x with some uncertainty. We wish to recover the hyperparameters of the distribution that generated the individual values of x.

[Text](http://google.com)


In [4]:
import numpy as np
import pymc

# define model hyperparameters
mu = 0.
sig = 1.

# specify the uncertainty on the individual measurements
err = 2.0

# pick the number of objects
nobj = 100

# generates values of x
x_true = np.random.normal(mu, sig, nobj)

# adds observational errors
x_obs = x_true + np.random.normal(0., err, nobj)

In [5]:
# infer mu, sig with an MCMC chain with Gibbs sampling.

mu_par = pymc.Uniform('mu', lower=-3., upper=3., value=mu)
sig_par = pymc.Uniform('sig', lower=0., upper=10., value=sig)

x_pars = [pymc.Normal('x_%d'%i, mu=mu_par, tau=1./sig_par**2, value=x_obs[i]) for i in range(nobj)]

hyperpars = [mu_par, sig_par]

# @pymc.deterministic()
# def likelihood(x=x_pars):
#    return (-0.5*(x - x_obs)**2/err**2 - np.log(err)).sum()

ps = [pymc.Normal('x_%d_obs'%i, mu=x_pars[i], tau=1./err**2, value=x_obs[i], observed=True) for i in range(nobj)]

#@pymc.stochastic()
#def logp(observed=True, value=0., h=hyperpars, x=x_pars):
#    return likelihood

M = pymc.MCMC(hyperpars + [x_pars])
#M.use_step_method(pymc.AdaptiveMetropolis, hyperpars)
M.sample(11000, 1000)

 [-----------------100%-----------------] 11000 of 11000 complete in 91.8 sec

In [6]:
# plots the chain and the posterior in mu, sig

import pylab
from scipy import ndimage

pylab.subplot(2, 2, 1)
pylab.plot(M.trace('mu')[:])

pylab.subplot(2, 2, 3)
pylab.plot(M.trace('sig')[:])

pylab.subplot(2, 2, 2)
for i in range(10):
    pylab.plot(M.trace('x_%d'%i)[:], color='gray')

pylab.subplot(2, 2, 4)
H, xbins, ybins = pylab.histogram2d(M.trace('mu')[:], M.trace('sig')[:], bins=100)
H = ndimage.gaussian_filter(H, 2)
sortH = np.sort(H.flatten())
cumH = sortH.cumsum()
lvl68 = sortH[cumH>cumH.max()*0.32].min()
lvl95 = sortH[cumH>cumH.max()*0.05].min()
lvl99 = sortH[cumH>cumH.max()*0.003].min()
contours = pylab.contour(H.T, [lvl68, lvl95, lvl99], colors='k', extent=(xbins[0], xbins[-1], ybins[0], ybins[-1]))
pylab.axvline(mu, linestyle='--', color='k')
pylab.axhline(sig, linestyle='--', color='k')
pylab.scatter(mu, sig)
pylab.show()